In [1]:
import numpy as np
import json

In [2]:
dataset = "symmetric"

In [3]:
path = "/home/xianglin/projects/DVI_data/noisy/{}/cifar10/embedding.npy".format(dataset)
embeddings = np.load(path)

path = "/home/xianglin/projects/DVI_data/noisy/{}/cifar10/clean_label.json".format(dataset)
with open(path, "r") as f:
    clean_label = json.load(f)
path = "/home/xianglin/projects/DVI_data/noisy/{}/cifar10/noisy_label.json".format(dataset)
with open(path, "r") as f:
    noisy_label = json.load(f)

clean_label = np.array(clean_label)
noisy_label = np.array(noisy_label)

In [ ]:
dataset = "pairflip"
path = "/home/xianglin/projects/DVI_data/resnet18_cifar10/embedding.npy"
embeddings = np.load(path)
path = "/home/xianglin/projects/DVI_data/noisy/{}/cifar10/clean_label.json".format(dataset)
with open(path, "r") as f:
    clean_label = json.load(f)
    
clean_label = np.array(clean_label)
noisy_label = np.copy(clean_label)

In [4]:
embeddings = embeddings.reshape(50000,-1,2)
embeddings.shape

(50000, 200, 2)

In [5]:
from noise_detector import NoiseTrajectoryDetector
ntd = NoiseTrajectoryDetector(embeddings, noisy_label)

In [7]:
# cls_num=2
# ntd.proj_cls(cls_num, period=75, repeat=2)
ntd.proj_all(period=75) # 75

Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...
Suspect abnormal in embedding...
Calculating umap scores...


In [ ]:
for cls_num in range(10):
    # print(cls_num, ntd.detect_noise_cls(cls_num))
    best_s, best_c = ntd.trajectory_eval[str(cls_num)]
    print("silhouette_score\t", best_s)
    print("calinski_harabasz_score\t", best_c)
    flag = best_c>.5

    print(cls_num, flag)

In [ ]:
for cls_num in range(10):
    # print(cls_num, ntd.detect_noise_cls(cls_num))
    best_s, best_c = ntd.trajectory_eval[str(cls_num)]
    print("silhouette_score\t", best_s)
    print("calinski_harabasz_score\t", best_c)
    flag = best_c>.5
    print(cls_num, flag)

In [ ]:
for num in range(10):
    ntd.show_ground_truth(num,clean_label[noisy_label==num])
    ntd.show_scores(num)

In [ ]:
for num in range(10):
    ntd.show_ground_truth(num,clean_label[noisy_label==num])
    ntd.show_scores(num)

In [ ]:
from noise_detector import select_closest
import math

for num in range(10):
    print("=================================={}=========================================".format(num))
    clean_labels = clean_label[noisy_label==num]
    ntd.show_ground_truth(num,clean_labels)

    c = ntd.sub_centers[str(num)]
    embedding = ntd.trajectory_embedding[str(num)]
    centroids = embedding[c]

    # vote for labels and scores
    c_labels = select_closest(embedding, centroids)
    centroid_labels = np.zeros(len(centroids))
    for i in range(len(centroids)):
        centroid_labels[i] = np.bincount(clean_labels[c_labels==i]).argmax()

    gt = centroid_labels
    nt = np.sum(gt!=num)
    ct = len(c) - nt

    curr_nt = nt
    flag = math.floor(nt*0.9)
    target_flag = flag

    for t in range(len(c)):
        id, _, selected = ntd.suggest_abnormal(num)
        d = False
        # if clean_label[noisy_label==num][id] != num:ß
        if gt[id] != num:
            d = True
            flag=flag-1
            curr_nt=curr_nt-1
        ntd.update_belief(num, selected, d)
        # ntd.show_verified(num)
        if flag==0:
            print("[{}]\t{} rounds find {} (90%) noise samples".format(target_flag/(t+1), t+1, target_flag))
            flag=10000
        if curr_nt==0:
            print("[{}]\t{} rounds find all {} noise samples".format(nt/(t+1), t+1, len(c)-ct))
            break

In [ ]:
from noise_detector import select_closest
import math

# Ground Truth
for num in range(10):
    print("=================================={}=========================================".format(num))
    clean_labels = clean_label[noisy_label==num]
    ntd.show_ground_truth(num,clean_labels)

    c = ntd.sub_centers[str(num)]
    embedding = ntd.trajectory_embedding[str(num)]
    centroids = embedding[c]

    # vote for labels and scores
    c_labels = select_closest(embedding, centroids)
    centroid_labels = np.zeros(len(centroids))
    for i in range(len(centroids)):
        centroid_labels[i] = np.bincount(clean_labels[c_labels==i]).argmax()

    gt = centroid_labels
    nt = np.sum(gt!=num)
    ct = len(c) - nt

    curr_nt = nt
    flag = math.floor(nt*0.9)
    target_flag = flag

    candidate_list = np.arange(len(c)).tolist()

    for t in range(len(c)):
        # id, _, selected = ntd.suggest_abnormal(num)
        id = np.random.choice(candidate_list,1)[0]
        selected = centroids[id]
        candidate_list.remove(id)
        d = False
        # if clean_label[noisy_label==num][id] != num:
        if gt[id] != num:
            d = True
            flag=flag-1
            curr_nt=curr_nt-1
        ntd.update_belief(num, selected, d)
        if flag==0:
            print("[{}]\t{} rounds find {} (90%) noise samples".format(target_flag/(t+1), t+1, target_flag))
            flag=10000
        if curr_nt==0:
            print("[{}]\t{} rounds find all {} noise samples".format(nt/(t+1), t+1, len(c)-ct))
            break